In [7]:
import pandas as pd

In [8]:
intrpr_df = pd.read_csv("../data/entry-matching-P0AGK8.tsv", sep="\t")
intrpr_df["range"] = intrpr_df.apply(lambda r: (int(r.Matches.split("..")[0]), int(r.Matches.split("..")[1])), axis=1)
intrpr_df["start pos"] = intrpr_df.apply(lambda r: r["range"][0], axis=1)
intrpr_df["end pos"] = intrpr_df.apply(lambda r: r["range"][1], axis=1)
intrpr_df

,Accession,Name,Source Database,Type,Integrated Into,Integrated Signatures,GO Terms,Protein Accession,Protein Length,Matches,range,start pos,end pos
0,G3DSA:1.10.10.10,Winged helix-like DNA-binding domain superfami...,cathgene3d,homologous_superfamily,IPR036388,NaN,NaN,p0agk8,162,1..144,"(1, 144)",1,144
1,IPR000944,Transcription regulator Rrf2,interpro,family,NaN,"PS51197,PF02082,TIGR00738,PTHR33221",NaN,p0agk8,162,1..148,"(1, 148)",1,148
2,IPR010242,"Transcription factor HTH, IscR",interpro,family,NaN,"MF_01176,TIGR02010","GO:0003690,GO:0003700,GO:0006355",p0agk8,162,1..162,"(1, 162)",1,162
3,IPR030489,"Transcription regulator Rrf2-type, conserved site",interpro,conserved_site,NaN,PS01332,NaN,p0agk8,162,49..67,"(49, 67)",49,67
4,IPR036388,Winged helix-like DNA-binding domain superfamily,interpro,homologous_superfamily,NaN,G3DSA:1.10.10.10,NaN,p0agk8,162,1..144,"(1, 144)",1,144
5,IPR036390,Winged helix DNA-binding domain superfamily,interpro,homologous_superfamily,NaN,SSF46785,NaN,p0agk8,162,2..133,"(2, 133)",2,133
6,MF_01176,HTH-type transcriptional regulator IscR [iscR],hamap,family,IPR010242,NaN,NaN,p0agk8,162,1..162,"(1, 162)",1,162
7,PF02082,Iron-dependent Transcriptional regulator,pfam,domain,IPR000944,NaN,NaN,p0agk8,162,3..132,"(3, 132)",3,132
8,PS01332,Rrf2-type HTH domain signature,prosite,conserved_site,IPR030489,NaN,NaN,p0agk8,162,49..67,"(49, 67)",49,67
9,PS51197,Rrf2-type HTH domain profile,profile,domain,IPR000944,NaN,NaN,p0agk8,162,2..131,"(2, 131)",2,131


In [9]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.gff import get_df_from_csv


def _update_gff_feature_description(feature, attribute):
    updated_feature = feature
    if "Note=Disordered" in attribute:
        updated_feature = "Disordered region"
    if "Note=Iron-sulfur (2Fe-2S)" in attribute:
        updated_feature = "2Fe-2S binding site"
    if "Note=H-T-H motif" in attribute:
        updated_feature = "HTH DNA binding domain"
    return updated_feature

gff_df = get_df_from_csv("./data/uniprot/iscR.gff")
gff_df = gff_df[~(gff_df["feature"].isin(["Sequence conflict", "Mutagenesis", "Beta strand", "Helix", "Turn",
#                                           "Chain"
                                         ]))]
# gff_df = gff_df[~(gff_df["attribute"].str.contains("Rho RNA-BD"))]
gff_df["feature"] = gff_df.apply(lambda r: _update_gff_feature_description(r["feature"], r["attribute"]), axis=1)
gff_df

,seqname,source,feature,start,end,score,strand,frame,attribute,9
0,P0AGK8,UniProtKB,Chain,1,162,.,.,.,ID=PRO_0000109560;Note=HTH-type transcriptiona...,NaN
1,P0AGK8,UniProtKB,Domain,2,131,.,.,.,Note=HTH rrf2-type,NaN
2,P0AGK8,UniProtKB,HTH DNA binding domain,28,51,.,.,.,Note=H-T-H motif;Ontology_term=ECO:0000250;evi...,NaN
3,P0AGK8,UniProtKB,Disordered region,140,162,.,.,.,Note=Disordered;Ontology_term=ECO:0000256;evid...,NaN
4,P0AGK8,UniProtKB,2Fe-2S binding site,92,92,.,.,.,Note=Iron-sulfur (2Fe-2S);Ontology_term=ECO:00...,NaN
5,P0AGK8,UniProtKB,2Fe-2S binding site,98,98,.,.,.,Note=Iron-sulfur (2Fe-2S);Ontology_term=ECO:00...,NaN
6,P0AGK8,UniProtKB,2Fe-2S binding site,104,104,.,.,.,Note=Iron-sulfur (2Fe-2S);Ontology_term=ECO:00...,NaN


In [10]:
def _get_barh_broken_range(r):
    return (r["start"], r["end"] - r["start"] + 1)

In [11]:
feats_df = pd.DataFrame(columns=["feature", "start", "end"])

for _, f in intrpr_df.iterrows():
    d = {"feature":f["Name"], "start":f["start pos"], "end":f["end pos"]}
    feats_df = feats_df.append(d, ignore_index=True)
    
for _, f in gff_df.iterrows():
    d = {"feature":f["feature"], "start":f["start"], "end":f["end"]}
    feats_df = feats_df.append(d, ignore_index=True)

    
# feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
# feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
feats_df

/tmp/ipykernel_20264/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_20264/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_20264/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_20264/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_20264/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed f

,feature,start,end,range
0,Winged helix-like DNA-binding domain superfami...,1,144,"(1, 144)"
1,Transcription regulator Rrf2,1,148,"(1, 148)"
2,"Transcription factor HTH, IscR",1,162,"(1, 162)"
3,"Transcription regulator Rrf2-type, conserved site",49,67,"(49, 19)"
4,Winged helix-like DNA-binding domain superfamily,1,144,"(1, 144)"
5,Winged helix DNA-binding domain superfamily,2,133,"(2, 132)"
6,HTH-type transcriptional regulator IscR [iscR],1,162,"(1, 162)"
7,Iron-dependent Transcriptional regulator,3,132,"(3, 130)"
8,Rrf2-type HTH domain signature,49,67,"(49, 19)"
9,Rrf2-type HTH domain profile,2,131,"(2, 130)"


In [12]:
interface_df = pd.read_csv('./data/mutfunc/iscR_interfaces.tab', sep='\t', header=None)
interface_df.columns = ["pdb","prot_a","prot_b","evidence","chain","aa_wt","aa_mt","pdb_pos","uniprot_pos","mut","dG_wt","dG_mt","ddG","dG_wt_sd","dG_mt_sd", "ddG_sd"]
# interface_df["NC_000913v3_pos"] = interface_df["uniprot_pos"] + 1  # Adding 1 to account for initiator methionine (start codon) within NC_000913v3 AA ref
interface_df.sort_values(by="uniprot_pos", inplace=True)
display(len(interface_df), interface_df.head())

1580

,pdb,prot_a,prot_b,evidence,chain,aa_wt,aa_mt,pdb_pos,uniprot_pos,mut,dG_wt,dG_mt,ddG,dG_wt_sd,dG_mt_sd,ddG_sd
1377,P0AGK8-P0AGK8-EXP-4hf0-pdb2-B-1-A-1.pdb,P0AGK8,P0AGK8,EXP,B,M,V,1,1,MB1V,-40.73116,-39.20316,1.52800,0.041067,0.035398,0.023213
577,P0AGK8-P0AGK8-EXP-4hf0-pdb2-B-1-A-1.pdb,P0AGK8,P0AGK8,EXP,A,M,V,1,1,MA1V,-40.88416,-38.74408,2.14008,0.018119,0.001842,0.017545
576,P0AGK8-P0AGK8-EXP-4hf0-pdb2-B-1-A-1.pdb,P0AGK8,P0AGK8,EXP,A,M,T,1,1,MA1T,-40.89258,-39.17558,1.71700,0.020079,0.012527,0.025574
575,P0AGK8-P0AGK8-EXP-4hf0-pdb2-B-1-A-1.pdb,P0AGK8,P0AGK8,EXP,A,M,S,1,1,MA1S,-40.89376,-38.03402,2.85974,0.023501,0.000045,0.023515
574,P0AGK8-P0AGK8-EXP-4hf0-pdb2-B-1-A-1.pdb,P0AGK8,P0AGK8,EXP,A,M,R,1,1,MA1R,-40.92152,-38.46026,2.46126,0.027052,0.168569,0.164595


In [13]:
df = interface_df.copy()
df = df.drop(columns=["aa_wt", "aa_mt", "mut", "dG_wt", "dG_mt", "ddG", "dG_wt_sd", "dG_mt_sd", "ddG_sd"]).drop_duplicates()
df.prot_b.unique()

array(['P0AGK8'], dtype=object)

In [14]:
from aledbmutil.gff import group_sequences

TODO: change gene_interface_df to something more general since it's so often reused

In [15]:
gene_interface_df = interface_df[((interface_df["prot_a"]=="P0AGK8") & (interface_df["prot_b"]=="P0AGK8"))]
# gene_interface_df = gene_interface_df["NC_000913v3_pos"].unique()
gene_interface_AAs = gene_interface_df["uniprot_pos"].unique()

# Outputting ranges for manual input into R
gene_interface_AAs.sort()
gene_interface_AA_ranges = group_sequences(gene_interface_AAs)
# gene_interface_AA_ranges

In [16]:
for rng_l in gene_interface_AA_ranges:
    feats_df = feats_df.append(
        {
            "feature": "IscR subunit interface",
            "start": rng_l[0],
            "end": rng_l[-1],
        },
        ignore_index=True
    )

display(len(feats_df), feats_df.head())

/tmp/ipykernel_20264/2096244761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_20264/2096244761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_20264/2096244761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_20264/2096244761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_20264/2096244761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.app

37

,feature,start,end,range
0,Winged helix-like DNA-binding domain superfami...,1,144,"(1, 144)"
1,Transcription regulator Rrf2,1,148,"(1, 148)"
2,"Transcription factor HTH, IscR",1,162,"(1, 162)"
3,"Transcription regulator Rrf2-type, conserved site",49,67,"(49, 19)"
4,Winged helix-like DNA-binding domain superfamily,1,144,"(1, 144)"


In [17]:
feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
display(len(feats_df), feats_df.head())

37

,feature,start,end,range
0,Disordered region,140,162,"(140, 23)"
1,"Transcription factor HTH, IscR",1,162,"(1, 162)"
2,HTH-type transcriptional regulator IscR [iscR],1,162,"(1, 162)"
3,Chain,1,162,"(1, 162)"
4,Transcription regulator Rrf2,1,148,"(1, 148)"


In [18]:
# https://www.rcsb.org/3d-sequence/4HF0?assemblyId=1
# https://www.rcsb.org/groups/sequence/polymer_entity/P0AGK8
l = [
    {'feature':'SO4 binding site', 'start':1, 'end':2},
    {'feature':'SO4 binding site', 'start':8, 'end':8},
    {'feature':'SO4 binding site', 'start':110, 'end':111},
    {'feature':'SO4 binding site', 'start':114, 'end':114},
]

df = pd.DataFrame(l)
df["range"] = df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
feats_df = pd.concat([feats_df, df])

In [19]:
feats_df.to_pickle('./annot_df.pkl')